pytorch関連のインストール

In [1]:
!pip3 install torch torchvision torchtext

    100% |████████████████████████████████| 484.0MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5c216000 @  0x7fceccad11c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 5.0MB/s 
    100% |████████████████████████████████| 51kB 17.9MB/s 
    100% |████████████████████████████████| 2.0MB 14.9MB/s 
    100% |████████████████████████████████| 51kB 15.8MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext


  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up gnupg-agent (2.1.15-1ubuntu8.1) ...
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up gnupg (2.1.15-1ubuntu8.1) 

In [3]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
path = "drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch"
os.chdir(path)

In [6]:
!pwd

/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch


In [0]:
import os
import sys
import time
import glob

import torch
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

from model import SNLIClassifier
from util import get_args, makedirs

In [5]:
sys.argv = [sys.argv[0], "--epochs", "10", '--d_embed', '300', "--word_vectors", "glove.6B.300d", '--vector_cache', '../input/vector_cache', '--train_embed']
args = get_args()
args
# !python train.py --epochs 5 --d_embed 300 --gpu -1 --word_vectors glove.6B.300d

Namespace(batch_size=128, birnn=True, d_embed=300, d_hidden=300, d_proj=300, dev_every=1000, dp_ratio=0.2, epochs=10, fix_emb=False, gpu=0, log_every=50, lower=True, lr=0.001, n_layers=1, projection=True, resume_snapshot='', save_every=1000, save_path='results', vector_cache='../input/vector_cache', word_vectors='glove.6B.300d')

In [0]:
torch.cuda.set_device(args.gpu)

In [0]:
inputs = data.Field(lower=args.lower)
answers = data.Field(sequential=False)
train, dev, test = datasets.SNLI.splits(inputs, answers, root="../input")

In [0]:
inputs.build_vocab(train, dev, test)

inputs.vocab.load_vectors(args.word_vectors, cache=args.vector_cache)
# if args.word_vectors:
#     if os.path.isfile(args.vector_cache):
#         inputs.vocab.vectors = torch.load(args.vector_cache)
#     else:
#         inputs.vocab.load_vectors(args.word_vectors)
#         makedirs(os.path.dirname(args.vector_cache))
#         torch.save(inputs.vocab.vectors, args.vector_cache)

answers.build_vocab(train)

In [0]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=args.batch_size, device=args.gpu)

In [0]:
config = args
config.n_embed = len(inputs.vocab)
config.d_out = len(answers.vocab)
config.n_cells = config.n_layers

# double the number of cells for bidirectional networks
if config.birnn:
    config.n_cells *= 2
    
criterion = nn.CrossEntropyLoss()

In [0]:
if args.resume_snapshot:
    model = torch.load(args.resume_snapshot, map_location=lambda storage, locatoin: storage.cuda(args.gpu))
else:
    model = SNLIClassifier(config)
    if args.word_vectors:
        model.embed.weight.data.copy_(inputs.vocab.vectors)
        model.cuda(args.gpu)

opt = O.Adam(model.parameters(), lr=args.lr)

In [12]:
iterations = 0
start = time.time()
best_dev_acc = -1
train_iter.repeat = False
header = '  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy'
dev_log_template = ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{:8.6f},{:12.4f},{:12.4f}'.split(','))
log_template =     ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{},{:12.4f},{}'.split(','))
makedirs(args.save_path)
print(header)

for epoch in range(args.epochs):
    train_iter.init_epoch()
    n_correct, n_total = 0, 0
    for batch_idx, batch in enumerate(train_iter):

        # switch model to training mode, clear gradient accumulators
        model.train(); opt.zero_grad()

        iterations += 1

        # forward pass
        answer = model(batch)

        # calculate accuracy of predictions in the current batch
        n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
        n_total += batch.batch_size
        train_acc = 100. * n_correct/n_total

        # calculate loss of the network output with respect to training labels
        loss = criterion(answer, batch.label)

        # backpropagate and update optimizer learning rate
        loss.backward(); opt.step()

        # checkpoint model periodically
        if iterations % args.save_every == 0:
            snapshot_prefix = os.path.join(args.save_path, 'snapshot')
            snapshot_path = snapshot_prefix + '_acc_{:.4f}_loss_{:.6f}_iter_{}_model.pt'.format(train_acc, loss.item(), iterations)
            torch.save(model, snapshot_path)
            for f in glob.glob(snapshot_prefix + '*'):
                if f != snapshot_path:
                    os.remove(f)

        # evaluate performance on validation set periodically
        if iterations % args.dev_every == 0:

            # switch model to evaluation mode
            model.eval(); dev_iter.init_epoch()

            # calculate accuracy on validation set
            n_dev_correct, dev_loss = 0, 0
            with torch.no_grad():
                for dev_batch_idx, dev_batch in enumerate(dev_iter):
                     answer = model(dev_batch)
                     n_dev_correct += (torch.max(answer, 1)[1].view(dev_batch.label.size()) == dev_batch.label).sum().item()
                     dev_loss = criterion(answer, dev_batch.label)
            dev_acc = 100. * n_dev_correct / len(dev)

            print(dev_log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), dev_loss.item(), train_acc, dev_acc))

            # update best valiation set accuracy
            if dev_acc > best_dev_acc:

                # found a model with better validation set accuracy

                best_dev_acc = dev_acc
                snapshot_prefix = os.path.join(args.save_path, 'best_snapshot')
                snapshot_path = snapshot_prefix + '_devacc_{}_devloss_{}__iter_{}_model.pt'.format(dev_acc, dev_loss.item(), iterations)

                # save model, delete previous 'best_snapshot' files
                torch.save(model, snapshot_path)
                for f in glob.glob(snapshot_prefix + '*'):
                    if f != snapshot_path:
                        os.remove(f)

        elif iterations % args.log_every == 0:

            # print progress message
            print(log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), ' '*8, n_correct/n_total*100, ' '*12))




  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy
     6     0        50    50/4292        1% 1.104479               33.3281             
    10     0       100   100/4292        2% 0.992439               35.4844             
    15     0       150   150/4292        3% 0.983435               39.4062             
    19     0       200   200/4292        5% 0.977478               42.3242             
    23     0       250   250/4292        6% 0.876767               44.7656             
    27     0       300   300/4292        7% 0.909086               47.0729             
    31     0       350   350/4292        8% 0.930804               49.0268             
    35     0       400   400/4292        9% 0.789268               50.5156             
    39     0       450   450/4292       10% 0.855933               51.6580             
    44     0       500   500/4292       12% 0.812234               52.6563             
    48     0       550   550/42

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


   101     0      1000  1000/4292       23% 0.809814 0.934278      58.4523      67.2628
   119     0      1050  1050/4292       24% 0.766774               58.8021             
   123     0      1100  1100/4292       26% 0.775671               59.1683             
   127     0      1150  1150/4292       27% 0.826599               59.5163             
   132     0      1200  1200/4292       28% 0.649009               59.8431             
   136     0      1250  1250/4292       29% 0.744398               60.1412             
   140     0      1300  1300/4292       30% 0.679232               60.4213             
   144     0      1350  1350/4292       31% 0.777790               60.7153             
   148     0      1400  1400/4292       33% 0.690305               60.9665             
   152     0      1450  1450/4292       34% 0.825880               61.2430             
   156     0      1500  1500/4292       35% 0.710802               61.4641             
   160     0      1550  1550/429

   369     0      3350  3350/4292       78% 0.649722               66.1504             
   373     0      3400  3400/4292       79% 0.651259               66.2309             
   377     0      3450  3450/4292       80% 0.642636               66.3075             
   381     0      3500  3500/4292       82% 0.744290               66.3879             
   385     0      3550  3550/4292       83% 0.561684               66.4683             
   390     0      3600  3600/4292       84% 0.663441               66.5438             
   394     0      3650  3650/4292       85% 0.724304               66.6173             
   398     0      3700  3700/4292       86% 0.572346               66.6829             
   402     0      3750  3750/4292       87% 0.815167               66.7479             
   406     0      3800  3800/4292       89% 0.740960               66.8185             
   410     0      3850  3850/4292       90% 0.650952               66.8933             
   414     0      3900  3900/429

   624     1      5700  1408/4292       33% 0.629736               75.7002             
   628     1      5750  1458/4292       34% 0.549249               75.7480             
   632     1      5800  1508/4292       35% 0.467365               75.7652             
   637     1      5850  1558/4292       36% 0.647852               75.8068             
   641     1      5900  1608/4292       37% 0.625064               75.8085             
   645     1      5950  1658/4292       39% 0.662606               75.8171             
   665     1      6000  1708/4292       40% 0.556866 0.805918      75.8325      74.6698
   683     1      6050  1758/4292       41% 0.728970               75.8390             
   687     1      6100  1808/4292       42% 0.489847               75.8599             
   691     1      6150  1858/4292       43% 0.495298               75.8481             
   695     1      6200  1908/4292       44% 0.588894               75.8550             
   700     1      6250  1958/429

...

  3348     7     31550  1506/4292       35% 0.403272               88.9676             
  3352     7     31600  1556/4292       36% 0.357347               88.9279             
  3356     7     31650  1606/4292       37% 0.177842               88.9263             
  3360     7     31700  1656/4292       39% 0.296881               88.9163             
  3364     7     31750  1706/4292       40% 0.272989               88.8931             
  3369     7     31800  1756/4292       41% 0.237181               88.8828             
  3373     7     31850  1806/4292       42% 0.305414               88.8588             
  3377     7     31900  1856/4292       43% 0.229636               88.8327             
  3381     7     31950  1906/4292       44% 0.345607               88.8116             
  3401     7     32000  1956/4292       46% 0.228736 0.815452      88.7957      76.6816
  3405     7     32050  2006/4292       47% 0.304188               88.7755             
  3409     7     32100  2056/429

  3574     7     33900  3856/4292       90% 0.283190               88.3015             
  3578     7     33950  3906/4292       91% 0.327442               88.2929             
  3604     7     34000  3956/4292       92% 0.276063 0.793169      88.2872      76.7324
  3608     7     34050  4006/4292       93% 0.442341               88.2750             
  3612     7     34100  4056/4292       95% 0.372902               88.2631             
  3616     7     34150  4106/4292       96% 0.425608               88.2512             
  3620     7     34200  4156/4292       97% 0.353831               88.2500             
  3624     7     34250  4206/4292       98% 0.348047               88.2268             
  3629     7     34300  4256/4292       99% 0.329480               88.2194             
  3635     8     34350    14/4292        0% 0.227532               90.0670             
  3639     8     34400    64/4292        1% 0.251807               90.2954             
  3643     8     34450   114/429

  3827     8     36250  1914/4292       45% 0.272541               89.9344             
  3831     8     36300  1964/4292       46% 0.279541               89.9229             
  3835     8     36350  2014/4292       47% 0.284902               89.9031             
  3839     8     36400  2064/4292       48% 0.233292               89.8782             
  3844     8     36450  2114/4292       49% 0.383706               89.8659             
  3848     8     36500  2164/4292       50% 0.242470               89.8456             
  3852     8     36550  2214/4292       52% 0.366634               89.8265             
  3856     8     36600  2264/4292       53% 0.355138               89.8192             
  3860     8     36650  2314/4292       54% 0.336676               89.8005             
  3864     8     36700  2364/4292       55% 0.235579               89.7909             
  3868     8     36750  2414/4292       56% 0.356335               89.7797             
  3872     8     36800  2464/429

  4053     8     38600  4264/4292       99% 0.278160               89.2355             
  4060     9     38650    22/4292        1% 0.235107               92.4006             
  4064     9     38700    72/4292        2% 0.175643               92.0247             
  4068     9     38750   122/4292        3% 0.263451               91.8097             
  4072     9     38800   172/4292        4% 0.272410               91.4971             
  4076     9     38850   222/4292        5% 0.266009               91.6104             
  4080     9     38900   272/4292        6% 0.250979               91.5728             
  4084     9     38950   322/4292        8% 0.258445               91.5567             
  4105     9     39000   372/4292        9% 0.228151 1.011792      91.5785      75.9602
  4109     9     39050   422/4292       10% 0.257300               91.5729             
  4113     9     39100   472/4292       11% 0.327634               91.5552             
  4117     9     39150   522/429

  4283     9     40950  2322/4292       54% 0.247398               90.6442             
  4303     9     41000  2372/4292       55% 0.275914 0.912562      90.6270      76.0821
  4307     9     41050  2422/4292       56% 0.284676               90.6166             
  4311     9     41100  2472/4292       58% 0.221801               90.6010             
  4315     9     41150  2522/4292       59% 0.198267               90.5832             
  4320     9     41200  2572/4292       60% 0.317021               90.5688             
  4324     9     41250  2622/4292       61% 0.198000               90.5618             
  4328     9     41300  2672/4292       62% 0.249431               90.5452             
  4332     9     41350  2722/4292       63% 0.281667               90.5360             
  4336     9     41400  2772/4292       65% 0.273156               90.5368             
  4340     9     41450  2822/4292       66% 0.253952               90.5298             
  4344     9     41500  2872/429

モデルを読み込んでテストデータで評価

In [23]:
path_model_eval = "results/best_snapshot_devacc_77.52489331436699_devloss_0.7100985050201416__iter_31000_model.pt"
model_eval = torch.load(path_model_eval, 
                        map_location=lambda storage, 
                        locatoin: storage.cuda(args.gpu))

def evaluate(model, data_iter, data_size):
    log_template = '{:>8.6f}, {:8.6f}, {:12.4f}%'
    model.eval(); data_iter.init_epoch()
  
    # calculate accuracy on validation set
    n_correct, loss = 0, 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_iter):
            answer = model(batch)
            n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
            loss = criterion(answer, batch.label)
    acc = 100. * n_correct / data_size

    print(log_template.format(loss.item(), loss.item(), acc))
        
evaluate(model_eval, test_iter, len(test))

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch/model.py:34: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  outputs, (ht, ct) = self.rnn(inputs, (h0, c0))


0.663549, 0.663549,      76.4047%


google driveとの同期プロセスも完全にkillする

In [0]:
# !kill -9 -1